Boston Housing Dataset Test
===========================

In this notebook we test revrand's ARD basis functions on the Boston housing dataset.

In [27]:
import numpy as np
from scipy.stats import gamma

from sklearn.datasets import load_boston
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

from revrand import StandardLinearModel
import revrand.basis_functions as bf
from revrand.metrics import smse, msll
from revrand.btypes import Parameter, Positive, Bound

# Log output to the terminal attached to this notebook
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")


In [28]:
# Load the data
boston = load_boston()
X = boston.data
y = boston.target - boston.target.mean()

N, D = X.shape
#lenscale = 10.
nbases = 50
#lenARD = lenscale * np.ones(D)
lenARD = gamma(10, scale=10)

In [29]:
# Construct basis functions
base = bf.RandomMatern32(Xdim=D, nbases=nbases, lenscale_init=Parameter(lenARD, Positive(), shape=(D,))) + \
    bf.LinearBasis(onescol=True)
#base = bf.spectralmixture(Xdim=D, bases_per_component=5, lenscales_init=[Parameter(l, Positive()) for l in lenARD]) + \
#    bf.LinearBasis(onescol=True)
#base = bf.SigmoidalBasis(centres=X[np.random.choice(N, 10), :], lenscale_init=Parameter(lenARD, Positive())) + \
#    bf.LinearBasis(onescol=True)
#base = bf.FastFoodRBF(Xdim=D, nbases=nbases, lenscale_init=Parameter(lenARD, Positive())) + \
#    bf.LinearBasis(onescol=True)
    

In [30]:
# Cross val
folds = 5
av_smse, av_msll, av_r2 = 0., 0., 0.

slm = StandardLinearModel(base)
foldgen = KFold(n_splits=folds, shuffle=True)

for i, (tr_ind, ts_ind) in enumerate(foldgen.split(X)):
    
    # Training
    slm.fit(X[tr_ind], y[tr_ind])
    
    # Prediction
    Ey, Vy = slm.predict_moments(X[ts_ind])

    # Validation
    f_smse = smse(y[ts_ind], Ey)
    f_msll = msll(y[ts_ind], Ey, Vy, y[tr_ind])
    f_r2 = r2_score(y[ts_ind], Ey)
    
    av_smse += f_smse
    av_msll += f_msll
    av_r2 += f_r2
    
    
    print("Fold: {},\n\trevrand: R2 = {}, SMSE = {}, MSLL = {}".format(i, f_r2, f_smse, f_msll))
    
av_smse /= folds
av_msll /= folds
av_r2 /= folds

Fold: 0,
	revrand: R2 = 0.8608783901477889, SMSE = 0.13912160985221111, MSLL = -1.0140701459227046
Fold: 1,
	revrand: R2 = 0.8176721928802634, SMSE = 0.1823278071197366, MSLL = -0.7045919191670094
Fold: 2,
	revrand: R2 = 0.7996052319718943, SMSE = 0.20039476802810569, MSLL = -0.8477514975629168
Fold: 3,
	revrand: R2 = 0.7503681542012091, SMSE = 0.24963184579879094, MSLL = -0.6804532620037262
Fold: 4,
	revrand: R2 = 0.8871939915252015, SMSE = 0.11280600847479845, MSLL = -1.142316310425567


In [31]:
# Print results
print("Final:\n\trevrand: R2 = {}, SMSE = {}, MSLL = {}".format(av_r2, av_smse, av_msll))


Final:
	revrand: R2 = 0.8231435921452714, SMSE = 0.1768564078547286, MSLL = -0.8778366270163849
